# Basics of GlyTrait

*GlyTrait version: 0.1.6*

*This notebook is a work in progress and is subject to change.*

**This notebook is an introduction to using GlyTrait as a Python package.**
**This provides more flexible functionalities and more control.**
**However, it needs a basic understanding of the Python language.**
**For the Command Line Interface, please refer to the [README](https://github.com/fubin1999/glytrait/blob/main/README.md).**
**For the Web app, just click [this link](https://glytrait.streamlit.app).**

## Introduction to derived traits

What are derived traits?
They are variables calculated from glycomics data that are more biologically relevant than glycan abundance.
As a glycomics researcher, you may have been working with derived traits already.
For example, you might wonder "What is the proportion of sialylated glycans in my samples?"
or "What is the ratio of complex glycans?".
These are derived traits.

You probably have developed your own methods to calculate these derived traits,
either using Excel or using code.
GlyTrait provides a more elegant way to calculate derived traits.
All you need to provide is glycan abundance and glycan structures,
and GlyTrait will do the rest 
(including preprocessing, calculating derived traits, filtering not useful derived traits, 
and even perform univariate statistical tests).
GlyTrait already integrates 100+ derived traits that are commonly used in N-glycomics.
What's better, by learning GlyTrait Formula Grammar, 
you can easily create your own derived traits.

## Basic usage of GlyTrait.

Please follow the installation guide in the [README](https://github.com/fubin1999/glytrait/blob/main/README.md) to install GlyTrait.

In [1]:
import pandas as pd
from glytrait import Experiment

The `Experiment` class is all you need.
You will create an instance of it and do all the steps by calling corresponding methods.
The detailed format instructions of the input files could be found in [README](https://github.com/fubin1999/glytrait/blob/main/README.md#input-file-format).
First, Let's create an instance.

In [2]:
# If you run this notebook in your environment, you might need to modify the file paths below.
exp = Experiment(
    abundance_file="../example_data/abundance.csv",
    glycan_file="../example_data/structures.csv",
)

If you want to check the input data, go for the `abundance_table` and `glycans` attributes.

In [3]:
exp.abundance_table.head()

,H3N3F0E0L0,H3N3F1E0L0,H3N4F0E0L0,H3N4F1E0L0,H3N5F1E0L0,H3N6F0E0L0,H4N3F0E0L0,H4N3F0E0L1,H4N3F0E1L0,H4N3F1E0L0,...,H7N6F0E3L1,H7N6F1E1L2,H7N6F1E1L3,H7N6F1E2L1,H7N6F1E2L2,H7N6F2E1L2,H7N6F2E1L3,H7N6F2E2L2,H8N2F0E0L0,H9N2F0E0L0
Sample,,,,,,,,,,,,,,,,,,,,,
S001,0.000297,0.000673,0.000468,0.035733,0.008853,0.000277,0.000489,0.000335,0.007382,0.000798,...,0.000652,0.001326,0.001816,0.000611,0.001642,0.000400,0.000581,0.000372,0.001214,0.001403
S030,0.000129,0.000178,0.000184,0.006099,0.002215,0.000176,0.000275,0.000288,0.006335,0.000351,...,0.001336,0.001250,0.001898,0.000594,0.001487,0.000307,0.000318,0.000185,0.002544,0.001686
S020,0.000596,0.001504,0.001390,0.106443,0.050744,0.000214,0.000677,0.000429,0.006335,0.001043,...,0.000243,0.001045,0.001265,0.000646,0.000482,0.000486,0.000320,0.000197,0.001681,0.003150
S038,0.000166,0.000191,0.000192,0.010799,0.004932,0.000143,0.000273,0.000280,0.007468,0.000249,...,0.000886,0.000393,0.000876,0.000136,0.000620,0.000177,0.000067,0.000079,0.001305,0.001923
A003,0.000254,0.000560,0.000933,0.028314,0.009947,0.000159,0.000507,0.000202,0.006691,0.000672,...,0.000758,0.000623,0.000576,0.000367,0.000543,0.000064,0.000174,0.000192,0.002372,0.003306


In [4]:
exp.glycans["H3N3F0E0L0"]

Structure(name='H3N3F0E0L0')

As you see, the `abundance_table` attribute is just a pandas DataFrame,
while the `glycans` attribute is a dict of glycan structures.
Don't worry about the latter.
It's something the `Experiment` instance will handle for you.l

Now let's run the whole workflow by calling the `run_workflow` method.
(The `corr_threshold` parameter will be explained later.)

In [5]:
exp.run_workflow(corr_threshold=0.9)

That's it!
Now we can access the results from the attributes.

After we calling `run_workflow`, the following attributes are available:

- `derived_trait_table`: all built-in derived traits;
- `filtered_derived_trait_table`: derived traits after removing those with too many missing values (this can happen), zero variance, or highly correlated with others;
- `meta_property_table`: the intermediate result GlyTrait uses to calculate derived traits;
- `processed_abundance_table`: glycan abundance after glycan filtering, imputation and normalization.

Now let's look at these results one by one.

In [6]:
exp.derived_trait_table.head()

,TM,THy,MHy,MM,CA1,CA2,CA3,CA4,CFc,A1Fc,...,A3F0S0Pl,A4Pl,A4FPl,A4F0Pl,A4SPl,A4S0Pl,A4FSPl,A4F0SPl,A4FS0Pl,A4F0S0Pl
Sample,,,,,,,,,,,,,,,,,,,,,
S001,0.008032,0.006020,1.334226,6.637587,0.011935,0.824623,0.146597,0.016845,0.346946,0.273635,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
S030,0.008745,0.003318,2.635396,7.166200,0.009399,0.668274,0.287619,0.034708,0.173764,0.240135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
S020,0.010453,0.006647,1.572592,7.169064,0.011980,0.803622,0.168459,0.015939,0.460645,0.315329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
S038,0.004888,0.004867,1.004414,7.707028,0.009449,0.779131,0.195171,0.016248,0.133322,0.122611,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
A003,0.009935,0.003908,2.542109,7.469993,0.010265,0.834835,0.143568,0.011332,0.297283,0.240878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [7]:
print(f"In total, {exp.derived_trait_table.shape[1]} derived traits were calculated")

In total, 152 derived traits were calculated


This is the most important result of GlyTrait.
Each column is a derived trait, and each row is a sample.

The definitions of each trait could be accessed from [to be added].
For example, `TM` is the proportion of high-mannose glycans,
and `CA2` is the proportion of bi-antennary glycans within complex glycans.

Note that there are some traits not that useful.
For example, some traits may be constant in all sample, 
others may be all NaN.

In [8]:
constant_columns = exp.derived_trait_table.columns[exp.derived_trait_table.nunique() == 1]
empty_columns = exp.derived_trait_table.columns[exp.derived_trait_table.isna().all()]

print("These traits are constants:", ", ".join(constant_columns))
print("These traits are all NaN:", ", ".join(empty_columns))

These traits are constants: A4S0Fc, A1Fa, A1SFa, A2SFa, A1S0Fa, A3S0Fa, A4S0Fa, A3B, A3FB, A3F0B, A3SB, A3S0B, A3FSB, A3F0SB, A3FS0B, A3F0S0B, A4B, A4FB, A4F0B, A4SB, A4S0B, A4FSB, A4F0SB, A4F0S0B, A4FG, A1SG, A4SG, A3S0G, A4S0G, A2FSG, A4FSG, A1F0SG, A2F0SG, A3F0SG, A4F0SG, A3FS0G, A3F0S0G, A4F0S0G, CPl, A2Pl, A2FPl, A2F0Pl, A2SPl, A2S0Pl, A2FSPl, A2F0SPl, A2FS0Pl, A2F0S0Pl, A3Pl, A3FPl, A3F0Pl, A3SPl, A3S0Pl, A3FSPl, A3F0SPl, A3FS0Pl, A3F0S0Pl, A4Pl, A4FPl, A4F0Pl, A4SPl, A4S0Pl, A4FSPl, A4F0SPl, A4F0S0Pl
These traits are all NaN: A4FS0B, A4FS0G, A4FS0Pl


We can filter out these traits manually.

In [9]:
traits_to_remove = set(constant_columns) | set(empty_columns)
traits_to_keep = set(exp.derived_trait_table.columns) - traits_to_remove
filtered_trait_table = exp.derived_trait_table[list(traits_to_keep)]
filtered_trait_table.shape

(595, 83)

This kind of filtering are easy to perform.

However, some derived traits are highly correlated with each other and have similar meanings.
For example:

In [10]:
filtered_trait_table["A4Fc"].corr(filtered_trait_table["A4SFc"])

0.9998740492963757

- `A4Fc`: The proportion of core-fucosylated glycans within all tetra-antennary glycans.
- `A4SFc`: The proportion of core-fucosylated glycans within all sialylated tetra-antennary glycans.

In this case, `A4SFc` seems redundant, for nearly all tetra-antennary glycans are sialylated.
Therefore, when performing downstream statistical analysis or machine learning, 
using `A4Fc` alone is enough.
This kind of situation is more complicated too be handled manually.
Luckily, GlyTrait performed a post-filtering step after derived trait calculation.
It removes all constant and empty traits,
followed by a collinearity filtering to remove redundant traits.
This result could be accessed from the `filtered_derived_trait_table` attribute.

In [11]:
exp.filtered_derived_trait_table.shape

(595, 63)

In [12]:
exp.filtered_derived_trait_table.head()

,TM,THy,MHy,MM,CA1,CA2,CA3,CA4,CFc,A1Fc,...,A1GS,A2GS,A3GS,A4GS,A1FGS,A2FGS,A3FGS,A4FGS,A2F0GS,A3F0GS
Sample,,,,,,,,,,,,,,,,,,,,,
S001,0.008032,0.006020,1.334226,6.637587,0.011935,0.824623,0.146597,0.016845,0.346946,0.273635,...,0.807309,0.773510,0.829995,0.855705,0.540992,0.445826,0.788973,0.913412,0.929455,0.868423
S030,0.008745,0.003318,2.635396,7.166200,0.009399,0.668274,0.287619,0.034708,0.173764,0.240135,...,0.899081,0.874054,0.762702,0.810342,0.761737,0.551666,0.800171,0.910961,0.943571,0.755415
S020,0.010453,0.006647,1.572592,7.169064,0.011980,0.803622,0.168459,0.015939,0.460645,0.315329,...,0.674610,0.568223,0.681592,0.806849,0.312047,0.207269,0.492196,0.877459,0.883414,0.843802
S038,0.004888,0.004867,1.004414,7.707028,0.009449,0.779131,0.195171,0.016248,0.133322,0.122611,...,0.905815,0.866215,0.919371,0.876856,0.615675,0.477692,0.765754,0.924821,0.924405,0.945417
A003,0.009935,0.003908,2.542109,7.469993,0.010265,0.834835,0.143568,0.011332,0.297283,0.240878,...,0.802265,0.777699,0.762666,0.796484,0.492621,0.390661,0.609346,0.896263,0.932297,0.854940


The filtering threshold (Pearson's correlation coefficient) 
was controlled by the `corr_threshold` parameter of the `run_workflow` method.
We set this to `0.9` before,
so any trait pairs with *r* above 0.9 will be judged and only the one with 
a more general interpretation will be kept.
In the case above, it's `A4Fc` that will be kept.

Fianlly, someone may find it useful to look at the `meta_property_table` attribute.
As mentioned above, this is the intermediate result GlyTrait uses to calculate derived traits.

In [13]:
exp.meta_property_table.head()

,type,B,nAnt,nF,nFc,nFa,nS,nM,nG,nN,Pl
H3N3F0E0L0,complex,False,1,0,0,0,0,3,0,3,False
H3N3F1E0L0,complex,False,1,1,1,0,0,3,0,3,False
H3N4F0E0L0,complex,False,2,0,0,0,0,3,0,4,False
H3N4F1E0L0,complex,False,2,1,1,0,0,3,0,4,False
H3N5F1E0L0,complex,True,2,1,1,0,0,3,0,5,False


As you can see, this is the characteristics of each glycan.
You may want to use this table for, say, drawing a heatmap with this properties as annotations,
or use this table for your own analysis.

That's all you need to know about the basics of how to use the Glytrait package.